# Quantitative Value Strategy

"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Import

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import math
import xlsxwriter
import requests
from scipy import stats

## Importing Our List of Stock and API Token

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio)

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 93451783, 'calculationPrice': 'close', 'change': 0.58, 'changePercent': 0.00381, 'close': 152.68, 'closeSource': 'oaifficl', 'closeTime': 1688747645609, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 154.51, 'delayedPriceTime': 1685181386005, 'extendedChange': -0.42, 'extendedChangePercent': -0.00271, 'extendedPrice': 154.04, 'extendedPriceTime': 1717894221634, 'high': 156.6, 'highSource': 'eiec r51pna ydedelumti ', 'highTime': 1689769450934, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 155.67, 'iexCloseTime': 1723821682628, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 153.132, 'iexOpenTime': 1688229393490, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1731981538815, 'latestPrice': 154.42, 'latestSource': 'Close', 'latestTime': 'November 18, 2022', 'latestUpdate': 1739681952015, 'latestVolume': 77146993, 'low': 150.05, 'lowSource': 'dci e1dr n5eim

## Parsing Our API Call

This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [symbol,
             data[symbol]['quote']['latestPrice'],
             data[symbol]['quote']['peRatio'],
             'N/A'],
            index = my_columns),
        ignore_index = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,152.78,34.9,N/A
1,AAL,14.08,-5.9,N/A
2,AAP,151.60,17.79,N/A
3,AAPL,156.04,25.21,N/A
4,ABBV,158.11,21.46,N/A
...,...,...,...,...
496,YUM,125.91,28.9,N/A
497,ZBH,118.57,87.4,N/A
498,ZBRA,259.60,28.87,N/A
499,ZION,51.70,5.83,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] >0] #removing negative values and 'none'
final_dataframe = final_dataframe[:50] #taking top 50 of the column after sorting
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index',axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AIG,62.14,3.68,N/A
1,PHM,44.01,4.39,N/A
2,AIV,8.10,4.46,N/A
3,PFG,92.10,4.51,N/A
4,APA,46.48,4.54,N/A
5,NUE,148.71,4.56,N/A
6,PVH,62.60,4.74,N/A
7,MOS,50.00,4.8,N/A
8,COF,102.69,4.95,N/A
9,PSX,113.57,5,N/A


## Calculating the Number of Shares to Buy

We now need to calculate the number of shares we need to buy.

To do this, we will use the portfolio_input function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        portfolio_size = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        portfolio_size = float(portfolio_size)

Use the portfolio_input function to accept a portfolio_size variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio: 1000000


You can now use the global portfolio_size variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = portfolio_size/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AIG,62.14,3.68,321
1,PHM,44.01,4.39,454
2,AIV,8.10,4.46,2469
3,PFG,92.10,4.51,217
4,APA,46.48,4.54,430
5,NUE,148.71,4.56,134
6,PVH,62.60,4.74,319
7,MOS,50.00,4.8,400
8,COF,102.69,4.95,194
9,PSX,113.57,5,176


## Building a Better (and More Realistic) Value Strategy

Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
- Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()


# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = ev/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gp = data[symbol]['advanced-stats']['grossProfit']
ev_to_gp = ev/gp

Let's move on to building our DataFrame. You'll notice that I use the abbreviation rv often. It stands for robust value, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-Earning Ratio', 'PE Percentile', 'Price-to-Book Ratio', 'PB Percentile', 
              'Price-to-Sales Ratio', 'PS Percentile', 'EV/EBITDA', 'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score']
rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gp = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = ev/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gp = ev/ebitda
        except TypeError:
            ev_to_gp = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [symbol,
                 data[symbol]['quote']['latestPrice'],
                 'N/A',
                 data[symbol]['quote']['peRatio'],
                 'N/A',
                 data[symbol]['advanced-stats']['priceToBook'],
                 'N/A',
                 data[symbol]['advanced-stats']['priceToSales'],
                 'N/A',
                 ev_to_ebitda,
                 'N/A',
                 ev_to_gp,
                 'N/A',
                 'N/A'
                ],
                index = rv_columns),
            ignore_index = True)
                
                 

In [13]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,146.46,N/A,34,N/A,8.6,N/A,6.8,N/A,24.946587,N/A,24.946587,N/A,N/A
1,AAL,14.31,N/A,-5.91,N/A,-1.17,N/A,0.208,N/A,20.775598,N/A,20.775598,N/A,N/A
2,AAP,154.50,N/A,17.85,N/A,3.29,N/A,0.8267,N/A,9.647355,N/A,9.647355,N/A,N/A
3,AAPL,156.82,N/A,24.92,N/A,49.6,N/A,6.4,N/A,19.302581,N/A,19.302581,N/A,N/A
4,ABBV,161.10,N/A,21.29,N/A,17.16,N/A,4.75,N/A,12.017009,N/A,12.017009,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.81,N/A,28.48,N/A,-4.16,N/A,5.32,N/A,20.949579,N/A,20.949579,N/A,N/A
497,ZBH,114.49,N/A,87.63,N/A,2.05,N/A,3.39,N/A,18.162172,N/A,18.162172,N/A,N/A
498,ZBRA,262.70,N/A,30.06,N/A,5.01,N/A,2.4,N/A,14.339139,N/A,14.339139,N/A,N/A
499,ZION,51.37,N/A,5.81,N/A,1.82,N/A,2.11,N/A,4.476935,N/A,4.476935,N/A,N/A


## Dealing With Missing Data in Our DataFrame

ur DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

You can use pandas' isnull method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,184.300,N/A,61.12,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,303.770,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,323.900,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,98.500,N/A,49.85,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,108.000,N/A,40.15,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,25.210,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,49.530,N/A,14.65,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,57.390,N/A,33.08,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,29.860,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


## Dealing with missing data is an important topic in data science.

There are two main approaches:

- Drop missing data from the data set (pandas' dropna method is useful here)
- Replace missing data with a new value (pandas' fillna method is useful here)

In this tutorial, we will replace missing data with the average non-NaN data point from that column.

Here is the code to do this:

In [15]:
for column in ['Price-to-Earning Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- EV/EBITDA
- EV/GP

Here's how we'll do this:

In [17]:
from scipy.stats import percentileofscore as score

metrics = {'Price-to-Earning Ratio' :'PE Percentile', 
           'Price-to-Book Ratio':'PB Percentile', 
           'Price-to-Sales Ratio':'PS Percentile', 
           'EV/EBITDA':'EV/EBITDA Percentile', 
           'EV/GP':'EV/GP Percentile'}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
rv_dataframe                                                             

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,146.46,N/A,34.00,0.782435,8.60,0.837325,6.8000,0.864271,24.946587,0.896208,24.946587,0.896208,N/A
1,AAL,14.31,N/A,-5.91,0.041916,-1.17,0.065868,0.2080,0.011976,20.775598,0.812375,20.775598,0.812375,N/A
2,AAP,154.50,N/A,17.85,0.401198,3.29,0.502994,0.8267,0.127745,9.647355,0.285429,9.647355,0.285429,N/A
3,AAPL,156.82,N/A,24.92,0.588822,49.60,0.98004,6.4000,0.850299,19.302581,0.772455,19.302581,0.772455,N/A
4,ABBV,161.10,N/A,21.29,0.49501,17.16,0.934132,4.7500,0.766467,12.017009,0.393214,12.017009,0.393214,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.81,N/A,28.48,0.680639,-4.16,0.061876,5.3200,0.802395,20.949579,0.818363,20.949579,0.818363,N/A
497,ZBH,114.49,N/A,87.63,0.966068,2.05,0.321357,3.3900,0.600798,18.162172,0.734531,18.162172,0.734531,N/A
498,ZBRA,262.70,N/A,30.06,0.714571,5.01,0.652695,2.4000,0.438124,14.339139,0.52495,14.339139,0.52495,N/A
499,ZION,51.37,N/A,5.81,0.091816,1.82,0.249501,2.1100,0.376248,4.476935,0.06986,4.476935,0.06986,N/A


## Calculating the RV Score

We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,146.46,N/A,34.00,0.782435,8.60,0.837325,6.8000,0.864271,24.946587,0.896208,24.946587,0.896208,0.855289
1,AAL,14.31,N/A,-5.91,0.041916,-1.17,0.065868,0.2080,0.011976,20.775598,0.812375,20.775598,0.812375,0.348902
2,AAP,154.50,N/A,17.85,0.401198,3.29,0.502994,0.8267,0.127745,9.647355,0.285429,9.647355,0.285429,0.320559
3,AAPL,156.82,N/A,24.92,0.588822,49.60,0.98004,6.4000,0.850299,19.302581,0.772455,19.302581,0.772455,0.792814
4,ABBV,161.10,N/A,21.29,0.49501,17.16,0.934132,4.7500,0.766467,12.017009,0.393214,12.017009,0.393214,0.596407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.81,N/A,28.48,0.680639,-4.16,0.061876,5.3200,0.802395,20.949579,0.818363,20.949579,0.818363,0.636327
497,ZBH,114.49,N/A,87.63,0.966068,2.05,0.321357,3.3900,0.600798,18.162172,0.734531,18.162172,0.734531,0.671457
498,ZBRA,262.70,N/A,30.06,0.714571,5.01,0.652695,2.4000,0.438124,14.339139,0.52495,14.339139,0.52495,0.571058
499,ZION,51.37,N/A,5.81,0.091816,1.82,0.249501,2.1100,0.376248,4.476935,0.06986,4.476935,0.06986,0.171457


## Selecting the 50 Best Value Stocks

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_dataframe.sort_values('RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace = True, drop = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GM,41.59,N/A,7.10,0.129741,0.8722,0.091816,0.3970,0.047904,2.162962,0.01996,2.162962,0.01996,0.061876
1,KSS,31.35,N/A,6.03,0.0998,0.8317,0.085828,0.1983,0.00998,4.144687,0.05988,4.144687,0.05988,0.063074
2,AIG,60.50,N/A,3.68,0.05988,1.1600,0.123752,0.7793,0.11976,2.055755,0.017964,2.055755,0.017964,0.067864
3,TSN,67.99,N/A,6.13,0.105788,0.9870,0.103792,0.3721,0.045908,3.804699,0.041916,3.804699,0.041916,0.067864
4,PHM,44.33,N/A,4.38,0.061876,1.2000,0.129741,0.6395,0.085828,3.598344,0.037924,3.598344,0.037924,0.070659
5,PVH,61.31,N/A,4.62,0.06986,0.7710,0.081836,0.4451,0.055888,4.716182,0.07984,4.716182,0.07984,0.073453
6,CCL,9.60,N/A,-1.58,0.053892,1.3000,0.140719,1.1000,0.181637,-14.276320,0.00998,-14.276320,0.00998,0.079242
7,COF,99.94,N/A,4.96,0.075848,0.7442,0.077844,1.0800,0.174651,3.596619,0.035928,3.596619,0.035928,0.08004
8,BA,174.02,N/A,-12.40,0.035928,-6.1200,0.057884,1.7100,0.298403,-27.977473,0.005988,-27.977473,0.005988,0.080838
9,UNM,42.88,N/A,7.01,0.125749,0.9603,0.097804,0.6894,0.101796,3.831181,0.043912,3.831181,0.043912,0.082635


## Calculating the Number of Shares to Buy

We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

Enter the value of your portfolio: 1000000


In [21]:
position_size = portfolio_size/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe

C:\Users\bison\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GM,41.59,480,7.10,0.129741,0.8722,0.091816,0.3970,0.047904,2.162962,0.01996,2.162962,0.01996,0.061876
1,KSS,31.35,637,6.03,0.0998,0.8317,0.085828,0.1983,0.00998,4.144687,0.05988,4.144687,0.05988,0.063074
2,AIG,60.50,330,3.68,0.05988,1.1600,0.123752,0.7793,0.11976,2.055755,0.017964,2.055755,0.017964,0.067864
3,TSN,67.99,294,6.13,0.105788,0.9870,0.103792,0.3721,0.045908,3.804699,0.041916,3.804699,0.041916,0.067864
4,PHM,44.33,451,4.38,0.061876,1.2000,0.129741,0.6395,0.085828,3.598344,0.037924,3.598344,0.037924,0.070659
5,PVH,61.31,326,4.62,0.06986,0.7710,0.081836,0.4451,0.055888,4.716182,0.07984,4.716182,0.07984,0.073453
6,CCL,9.60,2083,-1.58,0.053892,1.3000,0.140719,1.1000,0.181637,-14.276320,0.00998,-14.276320,0.00998,0.079242
7,COF,99.94,200,4.96,0.075848,0.7442,0.077844,1.0800,0.174651,3.596619,0.035928,3.596619,0.035928,0.08004
8,BA,174.02,114,-12.40,0.035928,-6.1200,0.057884,1.7100,0.298403,-27.977473,0.005988,-27.977473,0.005988,0.080838
9,UNM,42.88,466,7.01,0.125749,0.9603,0.097804,0.6894,0.101796,3.831181,0.043912,3.831181,0.043912,0.082635


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols. We'll need four main formats for our Excel document:

- String format for tickers
- XX.XX format for stock prices
- XX,XXX format for market capitalization
- Integer format for the number of shares to purchase
- Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [24]:
column_formats = {'A': ['Ticker', string_template],
                  'B': ['Price', dollar_template],
                  'C': ['Number of Shares to Buy', integer_template],
                  'D': ['Price-to-Earning Ratio', float_template],
                  'E': ['PE Percentile', percent_template],
                  'F': ['Price-to-Book Ratio', float_template],
                  'G': ['PB Percentile', percent_template],
                  'H': ['Price-to-Sales Ratio', float_template],
                  'I': ['PS Percentile', percent_template],
                  'J': ['EV/EBITDA', float_template],
                  'K': ['EV/EBITDA Percentile', percent_template],
                  'L': ['EV/GP', float_template],
                  'M': ['EV/GP Percentile', percent_template],
                  'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Outout

In [25]:
writer.save()